In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Any results you write to the current directory are saved as output.
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/recommendationsv4/meta_Grocery_and_Gourmet_Food.json
/kaggle/input/recommendationsv4/sample_submission.csv
/kaggle/input/recommendationsv4/test.csv
/kaggle/input/recommendationsv4/train.csv


In [2]:
train = pd.read_csv('/kaggle/input/recommendationsv4/train.csv')
test = pd.read_csv('/kaggle/input/recommendationsv4/test.csv')
submission = pd.read_csv('/kaggle/input/recommendationsv4/sample_submission.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
submission

,Id,rating
0,0,0.282813
1,1,0.629758
2,2,0.916899
3,3,0.702790
4,4,0.043446
...,...,...
285960,285960,0.049774
285961,285961,0.235110
285962,285962,0.525945
285963,285963,0.259647


In [4]:
submission

,Id,rating
0,0,0.282813
1,1,0.629758
2,2,0.916899
3,3,0.702790
4,4,0.043446
...,...,...
285960,285960,0.049774
285961,285961,0.235110
285962,285962,0.525945
285963,285963,0.259647


In [5]:
import scipy.sparse as sparse

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

import scipy.sparse as sparse

In [6]:
train_data, test_data = train_test_split(train,random_state=32, shuffle=True)


In [7]:
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

In [8]:
NUM_THREADS = 4 #число потоков
NUM_COMPONENTS = 30 #число параметров вектора 
NUM_EPOCHS = 20 #число эпох обучения

model = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS)
model = model.fit(ratings_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [9]:
preds = model.predict(test_data.userid.values,
                      test_data.itemid.values)


In [10]:
sklearn.metrics.roc_auc_score(test_data.rating,preds)


0.7464166987537848

In [11]:
preds = model.predict(test.userid.values,
                      test.itemid.values)


In [12]:
preds.min(), preds.max()

(-24.62344741821289, 23.222238540649414)

In [13]:
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()


In [14]:
normalized_preds.min(), normalized_preds.max()


(0.0, 1.0)

In [15]:
submission['rating']= normalized_preds

In [16]:
submission.to_csv('submission_log.csv', index=False)